## Introduction

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn import linear_model

In [27]:
#importing the tsv file as df
df=pd.read_csv('AmesHousing.tsv',delimiter='\t')

In [28]:
#defining functions to be used later

def transform_features(df):
    return df

def select_features(df):
    return df[['Gr Liv Area','SalePrice']]

def train_and_test(df):
    train=df[:1460]
    test=df[1460:]
    
    numeric_train=train.select_dtypes(include=['integer','float'])
    numeric_test=test.select_dtypes(include=['integer','float'])
    
    features=numeric_train.columns.drop('SalePrice')
    
    lr=linear_model.LinearRegression()
    lr.fit(numeric_train[features],numeric_train['SalePrice'])
    predictions=lr.predict(numeric_test[features])
    mse=mean_squared_error(numeric_test['SalePrice'],predictions)
    rmse=np.sqrt(mse)
    
    return rmse


In [29]:
transform_df=transform_features(df)
filtered_df=select_features(transform_df)
rmse=train_and_test(filtered_df)
rmse


57088.25161263909

### Feature Engineering
Handle missing values:

__All columns:__

* Drop any with 5% or more missing values for now. 

__Text columns:__

* Drop any with 1 or more missing values for now.

__Numerical columns:__

* For columns with missing values, fill in with the most common value in that column

1: All columns: Drop any with 5% or more missing values for now.

In [30]:
#finding missing values
num_missing=df.isnull().sum()
num_missing


Order                0
PID                  0
MS SubClass          0
MS Zoning            0
Lot Frontage       490
Lot Area             0
Street               0
Alley             2732
Lot Shape            0
Land Contour         0
Utilities            0
Lot Config           0
Land Slope           0
Neighborhood         0
Condition 1          0
Condition 2          0
Bldg Type            0
House Style          0
Overall Qual         0
Overall Cond         0
Year Built           0
Year Remod/Add       0
Roof Style           0
Roof Matl            0
Exterior 1st         0
Exterior 2nd         0
Mas Vnr Type        23
Mas Vnr Area        23
Exter Qual           0
Exter Cond           0
                  ... 
Bedroom AbvGr        0
Kitchen AbvGr        0
Kitchen Qual         0
TotRms AbvGrd        0
Functional           0
Fireplaces           0
Fireplace Qu      1422
Garage Type        157
Garage Yr Blt      159
Garage Finish      159
Garage Cars          1
Garage Area          1
Garage Qual

In [31]:
#removing columns with >5% missing values

df=df[num_missing[(num_missing<len(df)/20)].sort_values().index]

df.isnull().sum()

Order               0
Heating QC          0
Central Air         0
1st Flr SF          0
2nd Flr SF          0
Low Qual Fin SF     0
Gr Liv Area         0
Full Bath           0
Half Bath           0
Bedroom AbvGr       0
Kitchen AbvGr       0
Kitchen Qual        0
TotRms AbvGrd       0
Functional          0
Fireplaces          0
Paved Drive         0
Wood Deck SF        0
Open Porch SF       0
Enclosed Porch      0
3Ssn Porch          0
Screen Porch        0
Pool Area           0
Misc Val            0
Mo Sold             0
Yr Sold             0
Sale Type           0
Heating             0
Sale Condition      0
SalePrice           0
Year Built          0
                   ..
Exterior 2nd        0
Lot Config          0
Utilities           0
Exter Qual          0
Land Contour        0
Lot Shape           0
Exter Cond          0
Street              0
Lot Area            0
MS Zoning           0
MS SubClass         0
PID                 0
Land Slope          0
Condition 1         0
Garage Are

2: Text columns: Drop any with 1 or more missing values for now.

In [32]:
text_mv_counts=df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)

#dropping columns with missing values in text columns

df=df.drop(text_mv_counts[text_mv_counts>0].index,axis=1)

3: Numerical columns: For columns with missing values, fill in with the most common value in that column

In [33]:
#column-wise missing value counts
num_missing=df.select_dtypes(include=['int','float']).isnull().sum()
fixable_numeric_cols=num_missing[num_missing>0].sort_values()
fixable_numeric_cols

Garage Area        1
BsmtFin SF 2       1
Total Bsmt SF      1
Bsmt Unf SF        1
BsmtFin SF 1       1
Garage Cars        1
Bsmt Half Bath     2
Bsmt Full Bath     2
Mas Vnr Area      23
dtype: int64

In [34]:
# finding most common value for each column in 'fixable_numeric_cols' 
replacement_values_dict=df[fixable_numeric_cols.index].mode().to_dict(orient='records')[0]
replacement_values_dict

{'Bsmt Full Bath': 0.0,
 'Bsmt Half Bath': 0.0,
 'Bsmt Unf SF': 0.0,
 'BsmtFin SF 1': 0.0,
 'BsmtFin SF 2': 0.0,
 'Garage Area': 0.0,
 'Garage Cars': 2.0,
 'Mas Vnr Area': 0.0,
 'Total Bsmt SF': 0.0}

In [35]:
#filling the na values with replacement values
df=df.fillna(replacement_values_dict)

df.isnull().sum().value_counts()

0    64
dtype: int64

In [36]:
#creating features that better capture info in other features

years_sold=df['Yr Sold']-df['Year Built']
years_sold[years_sold<0]

2180   -1
dtype: int64

In [37]:
years_since_remod=df['Yr Sold']-df['Year Remod/Add']
years_since_remod[years_since_remod<0]


1702   -1
2180   -2
2181   -1
dtype: int64

In [38]:
#creating new columns
df['Years Before Sale']=years_sold
df['Years Since Remod']=years_since_remod

# dropping rows with negative values for both of the new features

df=df.drop([1702,2180,2181],axis=0)

#deleting original year columns
df=df.drop(["Year Built", "Year Remod/Add"],axis=1)


Dropping columns:
* Not useful for ML
* Lead data about final sale

In [39]:
#dropping columns not useful for model
df=df.drop(['PID','Order'],axis=1)

#dropping columns that leak info about final sale
df=df.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"],axis=1)

In [40]:
# Updating transform_features() function
def transform_features(df):
    num_missing=df.isnull().sum()
    drop_missing_cols=num_missing[(num_missing>len(df)/20)].sort_values()
    df=df.drop(drop_missing_cols.index,axis=1)
    
    text_mv_counts=df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
    drop_missing_cols_2=text_mv_counts[text_mv_counts>0]
    df=df.drop(drop_missing_cols_2.index,axis=1)
    
    num_missing=df.select_dtypes(include=['int','float']).isnull().sum()
    fixable_numeric_cols=num_missing[num_missing>0].sort_values()
    replacement_values_dict=df[fixable_numeric_cols.index].mode().to_dict(orient='records')[0]
    df=df.fillna(replacement_values_dict)
    
    years_sold=df['Yr Sold']-df['Year Built']
    years_since_remod=df['Yr Sold']-df['Year Remod/Add']
    df['Years Before Sale']=years_sold
    df['Years Since Remod']=years_since_remod
    df=df.drop([1720,2180,2181],axis=0)
    
    df = df.drop(["PID", "Order", "Mo Sold", "Sale Condition", "Sale Type", "Year Built", "Year Remod/Add"], axis=1)
    return df

def select_features(df):
    return df[['Gr Liv Area','SalePrice']]

def train_and_test(df):
    train=df[:1460]
    test=df[1460:]
    
    numeric_train=train.select_dtypes(include=['integer','float'])
    numeric_test=test.select_dtypes(include=['integer','float'])
    
    features=numeric_train.columns.drop('SalePrice')
    lr=linear_model.LinearRegression()
    lr.fit(train[features],train['SalePrice'])
    predictions=lr.predict(test[features])
    mse=mean_squared_error(test['SalePrice'],predictions)
    rmse=np.sqrt(mse)
    
    return rmse



In [41]:
#using the generated functions
df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

rmse

55272.304512350835

## Feature Selection

In [42]:
numerical_df = transform_df.select_dtypes(include=['int', 'float'])
numerical_df.head(5)

,MS SubClass,Lot Area,Overall Qual,Overall Cond,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,...,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Yr Sold,SalePrice,Years Before Sale,Years Since Remod
0,20,31770,6,5,112.0,639.0,0.0,441.0,1080.0,1656,...,62,0,0,0,0,0,2010,215000,50,50
1,20,11622,5,6,0.0,468.0,144.0,270.0,882.0,896,...,0,0,0,120,0,0,2010,105000,49,49
2,20,14267,6,6,108.0,923.0,0.0,406.0,1329.0,1329,...,36,0,0,0,0,12500,2010,172000,52,52
3,20,11160,7,5,0.0,1065.0,0.0,1045.0,2110.0,2110,...,0,0,0,0,0,0,2010,244000,42,42
4,60,13830,5,5,0.0,791.0,0.0,137.0,928.0,928,...,34,0,0,0,0,0,2010,189900,13,12


In [43]:
#finding correlation of other factors with target variable 'SalePrice'

abs_corr_coeff=numerical_df.corr()['SalePrice'].abs().sort_values()
abs_corr_coeff

BsmtFin SF 2         0.006063
Misc Val             0.019291
Yr Sold              0.030484
3Ssn Porch           0.032241
Bsmt Half Bath       0.035924
Low Qual Fin SF      0.037645
Pool Area            0.068415
MS SubClass          0.085306
Overall Cond         0.101635
Screen Porch         0.112202
Kitchen AbvGr        0.119787
Enclosed Porch       0.128744
Bedroom AbvGr        0.144107
Bsmt Unf SF          0.183227
Lot Area             0.267841
2nd Flr SF           0.269858
Bsmt Full Bath       0.276040
Half Bath            0.285294
Open Porch SF        0.316555
Wood Deck SF         0.327805
BsmtFin SF 1         0.438993
Fireplaces           0.474896
TotRms AbvGrd        0.498653
Mas Vnr Area         0.506683
Years Since Remod    0.535154
Full Bath            0.546208
Years Before Sale    0.559154
1st Flr SF           0.635324
Garage Area          0.641549
Total Bsmt SF        0.644175
Garage Cars          0.648333
Gr Liv Area          0.717713
Overall Qual         0.801382
SalePrice 

In [44]:
#keeping columns with correlation coeff larger than 0.4 (randomly chosen)

abs_corr_coeff[abs_corr_coeff>0.4]

BsmtFin SF 1         0.438993
Fireplaces           0.474896
TotRms AbvGrd        0.498653
Mas Vnr Area         0.506683
Years Since Remod    0.535154
Full Bath            0.546208
Years Before Sale    0.559154
1st Flr SF           0.635324
Garage Area          0.641549
Total Bsmt SF        0.644175
Garage Cars          0.648333
Gr Liv Area          0.717713
Overall Qual         0.801382
SalePrice            1.000000
Name: SalePrice, dtype: float64

In [45]:
#dropping columns with  <0.4 correation with 'SalePrice'

transform_df=transform_df.drop(abs_corr_coeff[abs_corr_coeff<0.4].index,axis=1)


In [46]:
#chosing the categorical columns
#list of categorical columns from documentation

nominal_features=["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]



* Need to encode the numerical columns as categorical where numbers don't have any semantic meaning.
* Need to tackle categorical column with hundreds of unique values/categories. As dummy code for this column would add hundreds of columns in the dataframe.

In [47]:
#taking the categorical columns from the dataframe we have cleaned
transform_cat_cols=[]
for col in nominal_features:
    if col in transform_df.columns:
        transform_cat_cols.append(col)
        
#finding unique values in each categorical col
uniqueness_counts=transform_df[transform_cat_cols].apply(lambda col:len(col.value_counts())).sort_values()

#Arbitrary cutoff of 10 unique values
drop_nonuniq_cols=uniqueness_counts[uniqueness_counts>10].index
transform_df=transform_df.drop(drop_nonuniq_cols,axis=1)

In [49]:
#selecting remaining text columns and converting to categorical
text_cols=transform_df.select_dtypes(include=['object'])

for col in text_cols:
    transform_df[col]=transform_df[col].astype('category')
    
#create dummy columns and add back to dataframe
transform_df=pd.concat([transform_df,pd.get_dummies(transform_df.select_dtypes(include=['category']))],axis=1).drop(text_cols,axis=1)


In [50]:
def transform_features(df):
    num_missing = df.isnull().sum()
    drop_missing_cols = num_missing[(num_missing > len(df)/20)].sort_values()
    df = df.drop(drop_missing_cols.index, axis=1)
    
    text_mv_counts = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
    drop_missing_cols_2 = text_mv_counts[text_mv_counts > 0]
    df = df.drop(drop_missing_cols_2.index, axis=1)
    
    num_missing = df.select_dtypes(include=['int', 'float']).isnull().sum()
    fixable_numeric_cols = num_missing[(num_missing < len(df)/20) & (num_missing > 0)].sort_values()
    replacement_values_dict = df[fixable_numeric_cols.index].mode().to_dict(orient='records')[0]
    df = df.fillna(replacement_values_dict)
    
    years_sold = df['Yr Sold'] - df['Year Built']
    years_since_remod = df['Yr Sold'] - df['Year Remod/Add']
    df['Years Before Sale'] = years_sold
    df['Years Since Remod'] = years_since_remod
    df = df.drop([1702, 2180, 2181], axis=0)

    df = df.drop(["PID", "Order", "Mo Sold", "Sale Condition", "Sale Type", "Year Built", "Year Remod/Add"], axis=1)
    return df

In [51]:
def select_features(df, coeff_threshold=0.4, uniq_threshold=10):
    numerical_df = df.select_dtypes(include=['int', 'float'])
    abs_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values()
    df = df.drop(abs_corr_coeffs[abs_corr_coeffs < coeff_threshold].index, axis=1)
    
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]
    
    transform_cat_cols = []
    for col in nominal_features:
        if col in df.columns:
            transform_cat_cols.append(col)

    uniqueness_counts = df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
    drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > 10].index
    df = df.drop(drop_nonuniq_cols, axis=1)
    
    text_cols = df.select_dtypes(include=['object'])
    for col in text_cols:
        df[col] = df[col].astype('category')
    df = pd.concat([df, pd.get_dummies(df.select_dtypes(include=['category']))], axis=1).drop(text_cols,axis=1)
    
    return df

In [55]:
def train_and_test(df, k=0):
    numeric_df = df.select_dtypes(include=['integer', 'float'])
    features = numeric_df.columns.drop("SalePrice")
    lr = linear_model.LinearRegression()
    
    if k == 0:
        train = df[:1460]
        test = df[1460:]

        lr.fit(train[features], train["SalePrice"])
        predictions = lr.predict(test[features])
        mse = mean_squared_error(test["SalePrice"], predictions)
        rmse = np.sqrt(mse)

        return rmse
    
    if k == 1:
        # Randomize *all* rows (frac=1) from `df` and return
        shuffled_df = df.sample(frac=1, )
        train = df[:1460]
        test = df[1460:]
        
        lr.fit(train[features], train["SalePrice"])
        predictions_one = lr.predict(test[features])        
        
        mse_one = mean_squared_error(test["SalePrice"], predictions_one)
        rmse_one = np.sqrt(mse_one)
        
        lr.fit(test[features], test["SalePrice"])
        predictions_two = lr.predict(train[features])   
        
        mse_two = mean_squared_error(train["SalePrice"], predictions_two)
        rmse_two = np.sqrt(mse_two)
        
        avg_rmse=np.mean([rmse_one,rmse_two])
        print(rmse_one)
        print(rmse_two)
        return(avg_rmse)
    
    else:
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        
        for train_index, test_index, in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            lr.fit(train[features], train["SalePrice"])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test["SalePrice"], predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
            
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        return avg_rmse
    

            
        
        
        
        

In [ ]:

df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df, k=4)

rmse